This notebook scrapes the top 1000 lifetime adjusted box office grosses from boxofficemojo.com. It exports the data as a list of dictionaries to 'mojo_cumulative_scraper_w_id.txt'.

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [ ]:
page_number_url_list = list(range(0, 1000, 200))
page_number_url_list

In [ ]:
mainpage_url_list = []

mainpage_url = \
"https://www.boxofficemojo.com/chart/top_lifetime_gross_adjusted/?adjust_gross_to=2020&offset="

for i in page_number_url_list:
    mainpage_url_list.append(mainpage_url + str(i))

mainpage_url_list

In [ ]:
list_of_each_movie_dict = []


for i in mainpage_url_list:
    mainpage = requests.get(i)
    soup = BeautifulSoup(mainpage.content, 'html.parser') 
    grandparent = soup.findAll('tr')

    for i in grandparent[1:]:
        x = i.find('a', {'class':'a-link-normal'}).get('href')
        id_slice_1 = str(x.split('/title/', 1)[1])
        id_slice_2 = str(id_slice_1.split('/?ref_', 1)[0])
        each_movie_mainpage_url = str('https://www.boxofficemojo.com' + x)
        
        movie_dict = {'adj_lifetime_gross': [],
                      'lifetime_gross': [],
                      'est_num_tickets': [],
                      'movie_name': [],
                      'production_budget':[],
                      'mpaa_rating':[],
                      'running_time': [],
                      'movie_genre': [],
                      'production_companies': [],
                      'earliest_dom_release_date': [],
                      'cast':[],
                      'movie_id': []
                     }

        movie_dict['adj_lifetime_gross'].append(i.findAll \
                    ('td', {'class': 'a-text-right mojo-field-type-money'})[0].get_text())        
        movie_dict['lifetime_gross'].append(i.findAll \
                    ('td', {'class': 'a-text-right mojo-field-type-money'})[1].get_text())
        movie_dict['est_num_tickets'].append(i.find \
                    ('td', {'class': 'a-text-right mojo-field-type-positive_integer'}) \
                    .get_text())
        
        summarypage = requests.get(each_movie_mainpage_url)
        soup2 = BeautifulSoup(summarypage.content, 'html.parser')


        try:
            production_company_string = (soup2.find('main').find('span', \
                    text = "Domestic Distributor").next_sibling.get_text()) \
                    .split('See full company information\n\n', 1)[0]
            movie_dict['production_companies'].append(production_company_string)
        except:
            pass

        try:
            movie_dict['movie_name'].append(soup2.find('main').find \
                    ('h1', {'class': 'a-size-extra-large'}).get_text())
        except:
            pass

        try:
            movie_dict['production_budget'].append(soup2.find('main') \
                    .find('span', text = "Budget").next_sibling.get_text())
        except:
            pass

        try:
            movie_dict['mpaa_rating'].append(soup2.find('main') \
                    .find('span', text = "MPAA").next_sibling.get_text())
        except:
            pass

        try:
            movie_dict['running_time'].append(soup2.find('main') 
                    .find('span', text = "Running Time").next_sibling.get_text())
        except:
            pass

        try:
            earliest_dom_release_date_string = (soup2.find('main') \
                    .find('span', text = "Earliest Release Date") \
                    .next_sibling.get_text()).split('\n', 1)[0]
            movie_dict['earliest_dom_release_date'] \
                    .append(earliest_dom_release_date_string)
        except:
            pass   

        try:
            movie_genre_string = (soup2.find('main').find('span', \
                    text = "Genres").next_sibling.get_text()) \
                    .split('\n    \n        ')
            movie_dict['movie_genre'].append(movie_genre_string)
        except:
            pass
        
        sep = '?'
        text = each_movie_mainpage_url
        rest = text.split(sep, 1)[0]
        crew_url_each_movie = (rest + 'credits')
        crewpage = requests.get(crew_url_each_movie)
        soup3 = BeautifulSoup(crewpage.content, 'html.parser')
        
        td_crew_list = soup3.find('table').findAll('tr')
        crew_member_dict = {}

        for j in td_crew_list[1:]:
            crew_name = j.get_text()
            v,k = crew_name.split('\n\n')
            if k in crew_member_dict:
                crew_member_dict[k].append(v)
            else:
                crew_member_dict[k] = [v]
        
        movie_dict.update(crew_member_dict)
        
        
        sep2 = '?'
        text2 = each_movie_mainpage_url
        rest2 = text.split(sep, 1)[0]
        cast_url_each_movie = (rest2 + 'credits')
        cast_list = []


        castpage = requests.get(cast_url_each_movie)
        soup4 = BeautifulSoup(castpage.content, 'html.parser')
        td_cast_list = soup4.find('table', {'id':'principalCast'}).findAll('a')


        for i in td_cast_list:
            cast_string = i.get_text().replace('\n\n', '')
            if cast_string != "See more":
                cast_list.append(cast_string)                
                
        movie_dict['cast'].append(cast_list)
        movie_dict['movie_id'].append(id_slice_2)
    
        print(movie_dict)

        list_of_each_movie_dict.append(movie_dict)
        
with open('mojo_cumulative_scraper_w_id.txt', 'w') as outfile:
    json.dump(list_of_each_movie_dict, outfile)
        
     


